# Tensor Mathematics & Circuit Composition
**Goals**

1. Illustrate how single‑qubit gates combine via the tensor product to act on larger registers.  
2. Demonstrate controlled operations (CNOT, controlled‑$U$, Toffoli) and inspect their unitaries.  
3. Verify equivalence between “matrix‑level” tensor products and:
    1. QISKIT 
    1. Cirq circuits.
    1. Pennylane QNodes.
    1. Julia QML

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1 Tensor product: $H \otimes I$

*H* on qubit 0, identity on qubit 1.

---
### QISKIT

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Operator, Statevector
from qiskit.visualization import plot_histogram, plot_bloch_vector

In [ ]:
# Build circuit: H on qubit 0, I on qubit 1
qc = QuantumCircuit(2)
qc.h(0)

# Circuit unitary
U_circ = Operator(qc).data

# Analytical Kronecker product
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
I = np.eye(2)
U_kron = np.kron(H, I)

print("Equal matrices? ", np.allclose(U_circ, U_kron))

---
### Cirq

In [ ]:
import cirq
from cirq import protocols

In [ ]:
# Two line qubits
q0, q1 = cirq.LineQubit.range(2)

# Circuit: H ⊗ I
tensor_circ = cirq.Circuit(cirq.H(q0))
U_circuit   = protocols.unitary(tensor_circ)

# Direct Kronecker product
H = np.array([[1, 1],
              [1,-1]]) / np.sqrt(2)
I = np.eye(2)
U_kron = np.kron(H, I)

print("Circuit unitary equals kron(H, I)?", np.allclose(U_circuit, U_kron))

---
### Pennylane

In [ ]:
import pennylane as qml

qml.about()

In [ ]:
dev = qml.device("default.qubit", wires=2, shots=None)

@qml.qnode(dev)
def kron_H_I():
    qml.Hadamard(wires=0)        # H ⊗ I
    return qml.state()

state = kron_H_I()

# Extract circuit unitary
U_circuit = qml.matrix(kron_H_I)()
print("Circuit unitary shape:", U_circuit.shape)

# Analytical Kronecker product
H = np.array([[1, 1], [1,-1]])/np.sqrt(2)
I = np.eye(2)
U_kron = np.kron(H, I)
print("Equal to kron(H, I)?", np.allclose(U_circuit, U_kron))

## 2 CNOT = controlled-$X$

---
### QISKIT:  Bell state via `H` + `CX`

In [ ]:
bell = QuantumCircuit(2, 2)
bell.h(0)
bell.cx(0, 1)
bell.measure([0, 1], [0, 1])

backend = Aer.get_backend("qasm_simulator")
counts = execute(bell, backend, shots=1024).result().get_counts()
plot_histogram(counts, title="Bell-state counts (|00⟩ and |11⟩)")

---
### Cirq

In [ ]:
cnot = cirq.Circuit(cirq.CNOT(q0, q1))
print("CNOT unitary:\n", protocols.unitary(cnot))

---
### Pennylane

In [ ]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def bell_state():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.state()

bell = bell_state()
qml.draw(bell_state, expansion_strategy="device")()

## 3 Controlled-$U$: example with $U = R_{y}(\pi/4)$

---
### QISKIT

In [ ]:
theta = np.pi / 4
qc_ctrl = QuantumCircuit(2)
qc_ctrl.cry(theta, 0, 1)   # controlled-R_y(π/4)

U_ctrl = Operator(qc_ctrl).data
print("Controlled-Ry matrix shape:", U_ctrl.shape)

---
### Cirq

In [ ]:
# Define single-qubit U
theta = np.pi/4
U = cirq.ry(theta)

# Controlled-U via cirq.ControlledGate
ctrl_U_gate = U.controlled()
ctrl_U = cirq.Circuit(ctrl_U_gate(q0, q1))

# Matrix form
print("Controlled-U matrix shape:", protocols.unitary(ctrl_U).shape)

---
### Pennylane

In [ ]:
theta = np.pi/4
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def ctrl_Ry():
    qml.ControlledQubitUnitary(qml.RY(theta, wires=1).matrix, control_wires=0, wires=1)
    return qml.state()

U_ctrl = qml.matrix(ctrl_Ry)()
print("Controlled-U matrix:\n", U_ctrl)

## 4 Toffoli (CCNOT) and decomposition

A three‑qubit gate built from single‑ and two‑qubit primitives.

---
### QISKIT: `CX` 

In [ ]:
toffoli = QuantumCircuit(3)
toffoli.ccx(0, 1, 2)

# Standard decomposition in Qiskit
decomp = toffoli.decompose()
decomp.draw('mpl')

---
### Cirq

In [ ]:
q2 = cirq.LineQubit(2)
toffoli = cirq.TOFFOLI(q0, q1, q2)

# Decompose Toffoli into native gates
decomp = cirq.Circuit(cirq.decompose_once(toffoli))
print("Decomposed circuit depth:", decomp.depth())
decomp

---
### Pennylane

Pennylane decomposes `Toffoli` into elementary rotations and `CNOTs` under the hood.

In [ ]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def toffoli_circuit():
    qml.Toffoli(wires=[0, 1, 2])
    return qml.state()

qml.draw(toffoli_circuit, expansion_strategy="device")()

### 5 Observations

* The Kronecker product `np.kron` reproduces the unitary for independent gates acting on separate qubits.  
* Controlled gates expand the dimension by a factor 2 per control qubit; 
    
    Cirq’s `controlled()` wrapper handles this transparently.  
* Multi‑qubit gates such as the Toffoli can be decomposed into one‑ and two‑qubit gates; 

    tensor products and control constructs let us scale small building‑blocks to full algorithms.

## 5 QISKIT Observations  

* The Kronecker product `np.kron` reproduces the unitary obtained from the circuit where gates act on separate qubits.  
* `CX` entangles two qubits after a Hadamard, yielding $|00⟩$ and $|11⟩$ with equal probability.  
* Controlled single‑qubit rotations (`cry`) expand to 4 × 4 matrices; their top‑left block is the identity, bottom‑right block is $R_y(\theta)$.  
* The three‑qubit Toffoli decomposes into single‑ and two‑qubit primitives, illustrating how large unitaries are built from a universal gate set.

## 5 Discussion

* The Kronecker product $\otimes$ (``np.kron``) reproduces multi‑qubit unitaries obtained from independent single‑qubit gates.  
* \texttt{ControlledQubitUnitary} adds one or more control wires to any single‑qubit unitary, extending dimension by $2^{n_\text{control}}$.  
* Complex gates such as $\texttt{Toffoli}$ are automatically decomposed into the native gate set, showing how small building blocks scale to larger registers.




### Further exercises

1. Verify that $(H \otimes H)\,\text{CNOT}\,(H \otimes H)$ implements a CZ gate.  
2. Construct a four‑qubit circuit that prepares the GHZ state and confirm its histogram shows only `0000` and `1111`.  
3. Replace the simulator with a real back‑end on:
    1. IBM when available. 
    1. Google Cloud Run when available.
    1. Pennylane on IBM via `qml.device("qiskit.ibmq", ...)}` when available.